<center>
<h1>Football Match Probability Prediction (Logistic Regression)</h1>
<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/34189/logos/header.png" />
</center>
<h4>This competition is about predicting the probabilities of more than 150000 match outcomes using the recent sequence of 10 matches of the teams.</h4>


<center>
Realisé Par :
ET-TOUIL Younes & 
DAKKON Abdelmonim
</center>

#### 1. Import modules and loading the data
First let's import modules, the training and the test set. The test set contains the same columns than the training set without the target.

In [102]:
# data processing
import pandas as pd
# linear algebra
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [103]:
# load the data
train = pd.read_csv("./Datasets/train.csv")
test = pd.read_csv("./Datasets/test.csv")

In [104]:
# show the first 5 rows
test.head()

,id,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,...,away_team_history_league_id_1,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8,away_team_history_league_id_9,away_team_history_league_id_10
0,17761448,team home,team away,2021-05-01 00:15:00,Division 1,755,False,9605704.0,30866250.0,2021-04-28 00:30:00,...,755.0,755.0,755.0,755.0,755.0,755.0,755.0,755.0,755.0,755.0
1,17695487,team home,team away,2021-05-01 00:30:00,Liga MX,743,False,10319425.0,460370.0,2021-04-24 02:06:00,...,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0
2,17715496,team home,team away,2021-05-01 01:00:00,Paulista A2,1314,False,440905.0,220998.0,2021-04-28 23:00:00,...,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1315.0,1315.0
3,17715493,team home,team away,2021-05-01 01:00:00,Paulista A2,1314,False,21036670.0,1553238.0,2021-04-29 01:00:00,...,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1746.0,1746.0
4,17715492,team home,team away,2021-05-01 01:00:00,Paulista A2,1314,False,3618158.0,37400444.0,2021-04-29 01:00:00,...,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0,1314.0


In [105]:
#function that return columns contient the string where we written
def features_filtred(data,column_name):
    features_filtred = [_ for _ in data.columns if str(column_name) in _]
    return features_filtred

#### 2. Preprocessing data
we drop all columns contient _id ,_name,_date

In [106]:
# we drop all columns contient _id ,_name,_date
#from train-datasets
all_id_features = features_filtred(train,"_id")
train = train.drop(all_id_features, 1)

all_name_features1 = features_filtred(train,"_name")
train = train.drop(all_name_features1, 1)

all_date_features2 = features_filtred(train,"date")
train = train.drop(all_date_features2, 1)

#from test-datasets
all_id_features3 = features_filtred(test,"_id")
test = test.drop(all_id_features3, 1)

all_name_features4 = features_filtred(test,"_name")
test = test.drop(all_name_features4, 1)

all_date_features5 = features_filtred(test,"date")
test = test.drop(all_date_features5, 1)



#### remove the missing values and replce them with its mean

In [107]:
#sum of missing values in every column in our datasets
train.isnull().sum()

id                                      0
target                                  0
is_cup                                  1
home_team_history_is_play_home_1     1159
home_team_history_is_play_home_2     2451
                                    ...  
away_team_history_coach_6           23525
away_team_history_coach_7           24377
away_team_history_coach_8           25219
away_team_history_coach_9           26150
away_team_history_coach_10          27129
Length: 143, dtype: int64

In [108]:
test.isnull().sum()

id                                      0
is_cup                                  0
home_team_history_is_play_home_1      498
home_team_history_is_play_home_2     1082
home_team_history_is_play_home_3     1678
                                    ...  
away_team_history_coach_6           20177
away_team_history_coach_7           20450
away_team_history_coach_8           20728
away_team_history_coach_9           20997
away_team_history_coach_10          21289
Length: 142, dtype: int64

In [109]:
#Replace the missing values in each column with its mean
train["is_cup"].fillna(True, inplace=True)
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [110]:
train.isnull().sum()

id                                  0
target                              0
is_cup                              0
home_team_history_is_play_home_1    0
home_team_history_is_play_home_2    0
                                   ..
away_team_history_coach_6           0
away_team_history_coach_7           0
away_team_history_coach_8           0
away_team_history_coach_9           0
away_team_history_coach_10          0
Length: 143, dtype: int64

In [111]:
train['target'].unique()

array(['away', 'home', 'draw'], dtype=object)

#### Encoding the two features ['target'] and ['is_cup']
where ['away', 'home', 'draw'] => [0, 2, 1] And ['True', 'False'] => [0, 1] 

In [112]:
#Encoding the two features ['target'] and ['is_cup']
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

test['is_cup']  = labelencoder.fit_transform(test['is_cup'])
train['is_cup'] = labelencoder.fit_transform(train['is_cup'])
train['target'] = labelencoder.fit_transform(train['target'])

In [113]:
train['is_cup'].unique()
test['is_cup'].unique()

array([0, 1], dtype=int64)

In [114]:
train['target'].unique()

array([0, 2, 1])

#### Split train data

In [115]:
#split train data to Input "X" and Output "Y"
ID = train['id']
Y = train['target']
X = train.iloc[:, 2:]

In [116]:
X

,is_cup,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,...,away_team_history_coach_1,away_team_history_coach_2,away_team_history_coach_3,away_team_history_coach_4,away_team_history_coach_5,away_team_history_coach_6,away_team_history_coach_7,away_team_history_coach_8,away_team_history_coach_9,away_team_history_coach_10
0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,468200.0,468200.0,4.682000e+05,4.682000e+05,4.682000e+05,4.682000e+05,4.682000e+05,4.682000e+05,4.682000e+05,4.682000e+05
1,0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,22169161.0,22169161.0,2.216916e+07,2.216916e+07,2.216916e+07,2.216916e+07,2.216916e+07,2.216916e+07,2.216916e+07,2.216916e+07
2,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,456313.0,456313.0,4.563130e+05,4.563130e+05,4.563130e+05,4.563130e+05,4.563130e+05,4.563130e+05,4.563130e+05,4.563130e+05
3,0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,465797.0,465797.0,4.657970e+05,4.657970e+05,4.657970e+05,4.657970e+05,4.657970e+05,4.657970e+05,4.657970e+05,4.657970e+05
4,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,426870.0,426870.0,4.268700e+05,4.268700e+05,4.268700e+05,4.268700e+05,4.268700e+05,3.726156e+07,3.726156e+07,3.726156e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110933,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,37564088.0,37564088.0,1.305275e+07,1.303644e+07,1.296347e+07,1.285662e+07,1.270277e+07,1.255002e+07,1.236828e+07,1.221457e+07
110934,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,16488543.0,16488543.0,1.648854e+07,1.648854e+07,1.648854e+07,4.718100e+05,4.718100e+05,4.718100e+05,4.718100e+05,4.718100e+05
110935,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,528562.0,528562.0,5.285620e+05,5.285620e+05,5.285620e+05,5.285620e+05,5.285620e+05,5.285620e+05,8.966820e+05,8.966820e+05
110936,0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,471757.0,471757.0,4.717570e+05,4.717570e+05,4.717570e+05,4.717570e+05,4.717570e+05,4.717570e+05,4.717570e+05,4.705150e+05


#### 3. Features Selection
The scikit-learn library provides the SelectKBest class which can be used with a suite of
different statistical tests to select a specific number of features.

In [117]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
SKB = SelectKBest(score_func=chi2, k=4)
fit = SKB.fit(X, Y)
np.set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)
print(features[0:5,:])
X.columns

[9.151e+01 7.244e+00 2.526e+00 3.250e+00 1.992e-01 1.756e+00 3.820e+00
 8.782e-03 4.019e+00 2.759e-01 4.881e-01 5.217e+01 4.293e+01 1.939e+01
 3.586e+01 2.884e+01 5.658e+01 4.985e+01 3.497e+01 5.600e+01 4.095e+01
 6.997e+02 6.633e+02 7.087e+02 5.057e+02 5.625e+02 5.171e+02 3.997e+02
 3.376e+02 3.472e+02 2.813e+02 9.979e+02 8.785e+02 8.396e+02 6.492e+02
 4.686e+02 6.186e+02 4.235e+02 4.058e+02 3.685e+02 3.558e+02 1.659e+03
 1.621e+03 1.472e+03 1.260e+03 1.314e+03 1.183e+03 1.068e+03 9.148e+02
 8.415e+02 7.893e+02 1.967e+03 1.734e+03 1.539e+03 1.358e+03 1.170e+03
 1.127e+03 9.740e+02 8.985e+02 8.523e+02 7.000e+02 1.590e+09 1.504e+09
 1.311e+09 1.199e+09 1.062e+09 1.039e+09 1.031e+09 9.016e+08 7.802e+08
 6.505e+08 8.667e-01 9.218e+00 1.180e+00 2.606e+00 7.605e-01 2.117e+00
 1.833e+00 2.116e+00 7.963e-01 2.647e-02 3.083e+01 1.928e+01 5.460e+01
 5.016e+01 6.034e+01 6.497e+01 3.024e+01 5.748e+01 3.717e+01 3.050e+01
 7.857e+02 6.025e+02 6.432e+02 6.155e+02 5.210e+02 4.853e+02 4.461e+02
 4.991

Index(['is_cup', 'home_team_history_is_play_home_1',
       'home_team_history_is_play_home_2', 'home_team_history_is_play_home_3',
       'home_team_history_is_play_home_4', 'home_team_history_is_play_home_5',
       'home_team_history_is_play_home_6', 'home_team_history_is_play_home_7',
       'home_team_history_is_play_home_8', 'home_team_history_is_play_home_9',
       ...
       'away_team_history_coach_1', 'away_team_history_coach_2',
       'away_team_history_coach_3', 'away_team_history_coach_4',
       'away_team_history_coach_5', 'away_team_history_coach_6',
       'away_team_history_coach_7', 'away_team_history_coach_8',
       'away_team_history_coach_9', 'away_team_history_coach_10'],
      dtype='object', length=141)

==> You can see the scores for each attribute and the chosen attributes (those with the highest scores).
higher)

#### Drop the features

In [118]:
# we drop the features with the low scores
all_history_coach_features = features_filtred(X,"history_coach_")
X = X.drop(all_history_coach_features, 1)

all_history_rating_features = features_filtred(X,"rating_")
X = X.drop(all_history_rating_features, 1)

all_history_coach_features_test = features_filtred(test,"history_coach_")
test = test.drop(all_history_coach_features_test, 1)

all_history_rating_features_test = features_filtred(test,"rating_")
test = test.drop(all_history_rating_features_test, 1)

#### Create sub-tables for features of Home team / Away team

In [119]:
#create sub-tables for features of Home team
home_team_history_is_play_home = features_filtred(train,"home_team_history_is_play_home")
X_HTPH = X[home_team_history_is_play_home]

home_team_history_is_cup = features_filtred(train,"home_team_history_is_cup_")
X_HTC = X[home_team_history_is_cup]

home_team_history_goal = features_filtred(train,"home_team_history_goal_")
X_HTG = X[home_team_history_goal]

home_team_history_opponent_goal = features_filtred(train,"home_team_history_opponent_goal")
X_HTOG = X[home_team_history_opponent_goal]


#create sub-tables for features of Away team

away_team_history_is_play_home = features_filtred(train,"away_team_history_is_play_home")
X_ATPH = X[away_team_history_is_play_home]

away_team_history_is_cup = features_filtred(train,"away_team_history_is_cup_")
X_ATC = X[away_team_history_is_cup]

away_team_history_goal = features_filtred(train,"away_team_history_goal_")
X_ATG = X[away_team_history_goal]

away_team_history_opponent_goal = features_filtred(train,"away_team_history_opponent_goal")
X_ATOG = X[away_team_history_opponent_goal]




In [120]:
#create sub-tables for features of Home team
home_team_history_is_play_home = features_filtred(test,"home_team_history_is_play_home")
X_HTPH_test = test[home_team_history_is_play_home]

home_team_history_is_cup = features_filtred(test,"home_team_history_is_cup_")
X_HTC_test = test[home_team_history_is_cup]

home_team_history_goal = features_filtred(test,"home_team_history_goal_")
X_HTG_test = test[home_team_history_goal]

home_team_history_opponent_goal = features_filtred(test,"home_team_history_opponent_goal")
X_HTOG_test = test[home_team_history_opponent_goal]


#create sub-tables for features of Away team

away_team_history_is_play_home = features_filtred(test,"away_team_history_is_play_home")
X_ATPH_test = test[away_team_history_is_play_home]

away_team_history_is_cup = features_filtred(test,"away_team_history_is_cup_")
X_ATC_test = test[away_team_history_is_cup]

away_team_history_goal = features_filtred(test,"away_team_history_goal_")
X_ATG_test = test[away_team_history_goal]

away_team_history_opponent_goal = features_filtred(test,"away_team_history_opponent_goal")
X_ATOG_test = test[away_team_history_opponent_goal]




In [121]:
X_HTPH_test

,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
72706,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
72707,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
72708,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
72709,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


#### 4. Principal Component Analysis
Principal component analysis uses linear algebra to transform the data set into a compressed form. Generally, this is called a data reduction technique. A property of PCA is that you can choose the number of dimensions
or principal component in the transformed result.
where we reduce the 10 features to one feature

In [122]:
from sklearn.decomposition import PCA

In [123]:
pca = PCA(n_components=1)

#PCA for Home_Team_Features
X_HTPH_C = pca.fit_transform(X_HTPH)
X_HTC_C = pca.fit_transform(X_HTC)
X_HTG_C = pca.fit_transform(X_HTG)
X_HTOG_C = pca.fit_transform(X_HTOG)

#PCA for Home_Team_Features
X_ATPH_C = pca.fit_transform(X_ATPH)
X_ATC_C = pca.fit_transform(X_ATC)
X_ATG_C = pca.fit_transform(X_ATG)
X_ATOG_C = pca.fit_transform(X_ATOG)



In [124]:
pca_test = PCA(n_components=1)

#PCA for Home_Team_Features
X_HTPH_C_test = pca_test.fit_transform(X_HTPH_test)
X_HTC_C_test = pca_test.fit_transform(X_HTC_test)
X_HTG_C_test = pca_test.fit_transform(X_HTG_test)
X_HTOG_C_test = pca_test.fit_transform(X_HTOG_test)

#PCA for Home_Team_Features
X_ATPH_C_test = pca_test.fit_transform(X_ATPH_test)
X_ATC_C_test = pca_test.fit_transform(X_ATC_test)
X_ATG_C_test = pca_test.fit_transform(X_ATG_test)
X_ATOG_C_test = pca_test.fit_transform(X_ATOG_test)

In [125]:
# create a new feature that replaces the 10 old features through PCA
X["home_team_is_play_home"] = X_HTPH_C
home_team_history_is_play_home = features_filtred(train,"home_team_history_is_play_home_")
X = X.drop(home_team_history_is_play_home,1)

X["home_team_is_cup"] = X_HTC_C
home_team_history_is_cup = features_filtred(train,"home_team_history_is_cup_")
X = X.drop(home_team_history_is_cup,1)

X["home_team_goal"] = X_HTG_C
home_team_history_goal = features_filtred(train,"home_team_history_goal_")
X = X.drop(home_team_history_goal,1)

X["home_team_opponent_goal"] = X_HTOG_C
home_team_history_opponent_goal = features_filtred(train,"home_team_history_opponent_goal")
X = X.drop(home_team_history_opponent_goal,1)



X["away_team_is_play_home"] = X_ATPH_C
away_team_history_is_play_home = features_filtred(train,"away_team_history_is_play_home_")
X = X.drop(away_team_history_is_play_home,1)

X["away_team_is_cup"] = X_ATC_C
away_team_history_is_cup = features_filtred(train,"away_team_history_is_cup_")
X = X.drop(away_team_history_is_cup,1)

X["away_team_goal"] = X_ATG_C
away_team_history_goal = features_filtred(train,"away_team_history_goal_")
X = X.drop(away_team_history_goal,1)

X["away_team_opponent_goal"] = X_ATOG_C
away_team_history_opponent_goal = features_filtred(train,"away_team_history_opponent_goal")
X = X.drop(away_team_history_opponent_goal,1)


In [126]:
# create a new feature that replaces the 10 old features through PCA
test["home_team_is_play_home"] = X_HTPH_C_test
home_team_history_is_play_home = features_filtred(test,"home_team_history_is_play_home_")
test = test.drop(home_team_history_is_play_home,1)

test["home_team_is_cup"] = X_HTC_C_test
home_team_history_is_cup = features_filtred(test,"home_team_history_is_cup_")
test = test.drop(home_team_history_is_cup,1)

test["home_team_goal"] = X_HTG_C_test
home_team_history_goal = features_filtred(test,"home_team_history_goal_")
test = test.drop(home_team_history_goal,1)

test["home_team_opponent_goal"] = X_HTOG_C_test
home_team_history_opponent_goal = features_filtred(test,"home_team_history_opponent_goal")
test = test.drop(home_team_history_opponent_goal,1)



test["away_team_is_play_home"] = X_ATPH_C_test
away_team_history_is_play_home = features_filtred(test,"away_team_history_is_play_home_")
test = test.drop(away_team_history_is_play_home,1)

test["away_team_is_cup"] = X_ATC_C_test
away_team_history_is_cup = features_filtred(test,"away_team_history_is_cup_")
test = test.drop(away_team_history_is_cup,1)

test["away_team_goal"] = X_ATG_C_test
away_team_history_goal = features_filtred(test,"away_team_history_goal_")
test = test.drop(away_team_history_goal,1)

test["away_team_opponent_goal"] = X_ATOG_C_test
away_team_history_opponent_goal = features_filtred(test,"away_team_history_opponent_goal")
test = test.drop(away_team_history_opponent_goal,1)


In [127]:
X

,is_cup,home_team_is_play_home,home_team_is_cup,home_team_goal,home_team_opponent_goal,away_team_is_play_home,away_team_is_cup,away_team_goal,away_team_opponent_goal
0,0,-1.441105,-0.264352,-0.471041,-1.071028,-0.200414,1.242825,0.536941,-1.343566
1,0,0.259370,-0.264352,-1.541345,-1.053435,0.785362,-0.263592,-2.002200,0.260212
2,0,-1.441105,-0.264352,1.596867,0.860644,0.220142,-0.263592,2.196798,-0.752953
3,0,-0.842933,-0.264352,1.395032,-0.176240,-0.127840,-0.010948,3.278611,2.031246
4,0,-1.441105,-0.264352,1.400139,0.099824,0.432988,-0.263592,1.211512,-1.181045
...,...,...,...,...,...,...,...,...,...
110933,0,-0.877736,-0.264352,0.921479,-1.478421,-0.385068,-0.263592,1.036548,0.152097
110934,0,-0.676428,0.390665,4.050365,0.098071,-0.322227,-0.263592,-0.342937,1.242329
110935,0,1.033658,0.217636,-1.214121,-0.997406,-0.773974,0.214686,-0.649378,-3.766883
110936,0,-0.644767,-0.264352,-1.284493,0.030306,0.432988,-0.263592,-2.254717,-2.117183


==> It can be seen that the transform dataset (9 principal components) bears little resemblance to the source data.

#### 5. Feature Importance , FI
Decision trees like Random Forest and Extra Trees can be used to estimate the importance of entities.
here we build a classifier ExtraTreesClassifier for the dataset. 

In [128]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

[0.004 0.13  0.078 0.144 0.146 0.13  0.078 0.145 0.145]


In [129]:
ID_test = test['id']
ID_train = ID
x_train = X
y_train = Y
x_test = test.iloc[:,1:]


#### 5. Training using Logistic Regression
Logistic regression is a classification algorithm used to assign observations
to a discrete set of classes

#### Fit the model and a make submission

In [130]:
from sklearn.linear_model import LogisticRegression

#using LogisticRegression for prediction
lg = LogisticRegression()
lg.fit(x_train, y_train)

LogisticRegression()

In [131]:
y_prop_pred = lg.predict_proba(x_test)
y_prop_pred


array([[0.152, 0.265, 0.583],
       [0.348, 0.313, 0.339],
       [0.266, 0.291, 0.444],
       ...,
       [0.535, 0.279, 0.186],
       [0.228, 0.259, 0.513],
       [0.305, 0.263, 0.432]])

In [132]:
submission_test = pd.DataFrame(y_prop_pred,columns=['away', 'draw', 'home'])
submission = pd.DataFrame()
submission['id'] = test[['id']]
submission['away'] = submission_test['away']
submission['draw'] = submission_test['draw']
submission['home'] = submission_test['home']


submission[['id', 'home', 'away', 'draw']].to_csv('./submission/submission.csv', index=False)

In [133]:

submission

,id,away,draw,home
0,17761448,0.151924,0.264612,0.583464
1,17695487,0.347603,0.313016,0.339380
2,17715496,0.265670,0.290603,0.443727
3,17715493,0.433550,0.307243,0.259207
4,17715492,0.272592,0.322197,0.405210
...,...,...,...,...
72706,18450246,0.289695,0.303449,0.406856
72707,18164889,0.185666,0.263527,0.550808
72708,18449018,0.535029,0.278639,0.186331
72709,17958831,0.227972,0.259303,0.512725


<h5> Kaggle_score :</h5>
<center>
<img src="https://user-images.githubusercontent.com/71790032/169912632-a8eda7be-8d3b-4b21-944a-20b860193cfd.jpeg" />
</center>